# Importando bibliotecas e arquivos

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from dataprep import bookfeat,tradefeat,max_min, aggfunc
from modelo import modelo

## Lendo o endereço de cada dataframe

In [ ]:
df_url=pd.read_csv('./Projeto 3 - Volatilidade/df_url.csv')
df_url

In [ ]:
a=0
for i in df_url['stock_id'].unique():
    df_url['trade'][a]=(f'./Projeto 3 - Volatilidade/trade_train.parquet/stock_id={i}/' + df_url['trade'][a])
    df_url['book'][a]=(f'./Projeto 3 - Volatilidade/trade_train.parquet/stock_id={i}/' + df_url['book'][a])
    a+=1
    

# Agregando Dataframe

Looping para abrir cada arquivo do tipo .parquet que compõe os dataframes de book e trade para cada ação. Dentro do loop realizar os features engineering para ambos os dataframes e fazer as agregações necessárias para alcançar a granularidade do dataframe que vai para o modelo com o target.

In [ ]:
for i in tqdm(df_url['stock_id'].unique()):
    x = df_url.loc[df_url['stock_id']==i,'book'].values[0]
    y = df_url.loc[df_url['stock_id']==i,'trade'].values[0]
    df_book = bookfeat(pd.read_parquet(x))
    df_trade = tradefeat(pd.read_parquet(y))
    
    df_final1=aggfunc(df_book,df_trade)
    df_final1['stock_id'] = i
    df_final=pd.concat([df_final,df_final1])
    

Limpando e editando o dataframe gerado

In [ ]:
df_final = df_final.dropna()

In [ ]:
df_final=df_final.drop_duplicates().reset_index()


In [ ]:
df_final['stock_id']=df_final['stock_id'].astype(float)
df_final['stock_id']=df_final['stock_id'].astype(int)

In [ ]:
df_final['stock_id']=df_final['stock_id'].astype(str)
df_final['time_id']=df_final['time_id'].astype(str)

Criando um identificador que agrega o número da ação com o time_id dela

In [ ]:
df_final['id'] = 't'+df_final['time_id'] +'_st'+df_final['stock_id']

Abrindo o dataframe com o target e criando o mesmo identificador criado anteriormente para juntar os dataframes

In [ ]:
df_target=pd.read_csv(r'C:\Users\guede\projetos\Projeto 3 - Volatilidade/train.csv')
df_target

In [ ]:
df_target['stock_id']=df_target['stock_id'].astype(str)
df_target['time_id']=df_target['time_id'].astype(str)

In [ ]:
df_target['id'] = 't'+df_target['time_id'] +'_st'+df_target['stock_id']

Agregando os dataframes

In [ ]:
df_merge = pd.merge(df_target, df_final, on='id')

In [ ]:
df_merge = df_merge.drop_duplicates()

Separando os dataframes para botar no modelo

In [ ]:
df_target_new = df_merge[['stock_id_x', 'time_id_x', 'target']]
df_prep_new = df_merge[['time_id_y',
       'wap_std', 'wap_mean', 'wap_max_min', 'wap2_std', 'wap2_mean',
       'wap2_max_min', 'seconds_in_bucket_std', 'seconds_in_bucket_mean',
       'seconds_in_bucket_max_min', 'log_return_ask_std',
       'log_return_ask_mean', 'log_return_ask_max_min', 'log_return_bid_std',
       'log_return_bid_mean', 'log_return_bid_max_min', 'log_ask_div_bid_std',
       'log_ask_div_bid_mean', 'log_ask_div_bid_max_min',
       'log_vol_ask_div_bid_std', 'log_vol_ask_div_bid_mean',
       'log_vol_ask_div_bid_max_min', 'log_ask_div_ask_std',
       'log_ask_div_ask_mean', 'log_ask_div_ask_max_min',
       'log_bid_div_bid_std', 'log_bid_div_bid_mean',
       'log_bid_div_bid_max_min', 'log_vol_ask_div_ask_std',
       'log_vol_ask_div_ask_mean', 'log_vol_ask_div_ask_max_min',
       'log_vol_bid_div_bid_std', 'log_vol_bid_div_bid_mean',
       'log_vol_bid_div_bid_max_min', 'acima/abaixo media_sum', 'tend_vol_sum',
       'intervalo_bucket_std', 'intervalo_bucket_mean',
       'intervalo_bucket_max_min', 'price*size/order_count*intervalo_std',
       'price*size/order_count*intervalo_mean',
       'price*size/order_count*intervalo_max_min', 'size/order_count_std',
       'size/order_count_mean']]

In [ ]:
df_target_new=df_target_new.rename(columns = {'stock_id_x': 'stock_id', 'time_id_x':'time_id'})
df_prep_new = df_prep_new.rename( columns={ 'time_id_y':'time_id'})

# Aplicação do modelo 

In [ ]:
target,rmspe,grid_fit,final_list = modelo(df_prep_new,df_target_new)

In [ ]:
rmspe

Exportando o dataframe para salvar as informações

In [ ]:
target = pd.read_csv(r'./Projeto 3 - Volatilidade/target_final.csv')
target

Abrindo a biblioteca Shap e utilizando as funções para visualização 

In [ ]:
import shap

In [ ]:
explainer = shap.KernelExplainer(model = params.predict, data =df_prep_new[:428]  ,link = "identity")

In [ ]:
X,y = (df_prep_new[final_list][:428] , df_target_new['pred_boosting'][:428])
explainer = shap.TreeExplainer(params)
shap_values = explainer.shap_values(X)


In [ ]:
# visualize the first prediction's explanation
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X)

In [ ]:
shap.summary_plot(shap_values, X)

Visualização da dependência de cada uma das variáveis

In [ ]:
shap.dependence_plot("intervalo_bucket_std", shap_values, X)

## Avaliando os erros individuais 

In [ ]:
def rmspe_stock_id(df):
    lista=[]
    lista_id =[]
    for i in df['stock_id'].unique():
        df_use= df[df['stock_id']==i]
        rmspe = (np.sqrt(np.mean(np.square((df_use["target"] - df_use["pred_boosting"]) / df_use["target"]))))
        lista.append(rmspe)
        lista_id.append(i)
    return lista,lista_id

In [ ]:
lista_rmspe,lista_id=rmspe_stock_id(df_target_new)

In [ ]:
df_rmspe = pd.DataFrame({'Stock_id':lista_id,'RMSPE':lista_rmspe})
df_rmspe.sort_values('RMSPE',ascending = True)